In [3]:
from us import states
import csv
import requests
import pandas as pd
import pdb

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 1000)


import googlemaps

# --- Use US Government Census API (http://api.census.gov/data/key_signup.html)
# --- Use Google API to fetch coordinates of places
# --- The API key are user-specific and stored in your own file API_keys.py in the variables
# --- GOOGLE_API_KEY and CENSUS_API_KEY
execfile('./API_keys.py')
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

height has been deprecated.



## Get Data from US Census

In [682]:
# --- Codes in the US Governmental Census API
# --- Data to be fetched in http://api.census.gov/data/2013/acs5/variables.html
# --- More data could be fed by using the right Census Code.
# B01003_001E - Total Population
# B02001_002E - White alone
# B11001_001E -  Households Type (Number of household)
# B06011_001E - Median income in the past 12 months --!!Total:
# B19083_001E - Gini index
# C24070_001E - Industry by Class of Worker for the Civilian Employed Population 16 Years and Over
# C24070_002E - Industry by Class of Worker in agriculture, forestry, fishing, hunting, mining

features =      ["B01003_001E","B02001_002E","B11001_001E","B06011_001E","B19083_001E","C24070_001E","C24070_002E"]
feature_names = ["Total Population","White Population","Number of Household",
                 "Median Income","Gini Index","Industry Total","Industry in Agriculture"]

In [683]:
def fetch_data_year(year):
    
    geoLimit='&for=place:*&in=state:' + states.CA.fips + '&key=' + API_KEY
    api_url_base = 'http://api.census.gov/data/'+str(year)+'/acs5?get=NAME,'
    api_url=api_url_base+','.join(features)+geoLimit+'&key='+API_KEY
    r = requests.get(api_url)
    data=r.json()
    
    df = pd.DataFrame(data[1:],columns = ["City Name"] + feature_names + ["State","Place"])
    df.drop("State",1, inplace=True)
    df.drop("Place",1, inplace=True)
    df = df.convert_objects(convert_numeric = True)

    df["White Proportion (in %)"] = df["White Population"]/df["Total Population"] * 100
    df.drop("White Population", 1, inplace = True)

    df["Household Size"] = df["Total Population"]/df["Number of Household"]
    df.drop("Number of Household", 1, inplace = True)
    df["Agriculture Employement (in %)"] = df["Industry in Agriculture"]/df["Industry Total"] * 100
    df.drop("Industry in Agriculture", 1, inplace = True)
    df.drop("Industry Total", 1, inplace = True)
    
    for city_index in range(len(df)):
        df.iloc[city_index,0] = df.iloc[city_index,0][:-12]
    for city_index in range(len(df)):
        if df.iloc[city_index,0] == u'La Ca\xf1ada Flintridge city':
            df.iloc[city_index,0] = 'La Canada Flintridge'
            continue
        if df.iloc[city_index,0] == u'Pi\xf1on Hills CDP':
            df.iloc[city_index,0] = 'Pinon Hills'
            continue
        if df.iloc[city_index,0][-4:] == " CDP":
            df.iloc[city_index,0] = str(df.iloc[city_index,0][:-4])
        if df.iloc[city_index,0][-5:] == " city":
            df.iloc[city_index,0] = str(df.iloc[city_index,0][:-5])
    return df

In [679]:
df_2013 = fetch_data_year(2013)
#df_2012 = fetch_data_year(2012)
#df_2011 = fetch_data_year(2011)
#df_2010 = fetch_data_year(2010)

In [ ]:
df_2013.loc[df_2013["City Name"] == "Jurupa Valley","City Name"] = "Jurapa"
df_2013.loc[df_2013["City Name"] == "Orangevale","City Name"] = "Orange Vale"
df_2013.loc[df_2013["City Name"] == "Dublin","City Name"] = "Dublin San Ramon"
df_2013.loc[df_2013["City Name"] == "El Dorado Hills","City Name"] = "El Dorado"
df_2013.loc[df_2013["City Name"] == "San Lorenzo","City Name"] = "San Lorenzo Valley"
df_2013.loc[df_2013["City Name"] == "South Lake Tahoe","City Name"] = "South Tahoe"
df_2013.loc[df_2013["City Name"] == "Lake Elsinore","City Name"] = "Elsinore Valley"
df_2013.loc[df_2013["City Name"] == "Coachella","City Name"] = "Coachella Valley"
df_2013.loc[df_2013["City Name"] == "Contra Consta Center","City Name"] = "Contra Consta"
df_2013.loc[df_2013["City Name"] == "Antelope","City Name"] = "Antelope Valley"
df_2013.loc[df_2013["City Name"] == "Irvine","City Name"] = "Irvine Ranch"
df_2013.loc[df_2013["City Name"] == "Rowland Heights","City Name"] = "Rowland"
df_2013.loc[df_2013["City Name"] == "South Lake Tahoe","City Name"] = "South Tahoe"
df_2013.loc[df_2013["City Name"] == "Desert Center","City Name"] = "South Tahoe"
df_2013.loc[df_2013["City Name"] == "South Lake Tahoe","City Name"] = "South Tahoe"
df_2013.loc[df_2013["City Name"] == "Desert Center","City Name"] = "Hi-Desert"
df_2013.loc[df_2013["City Name"] == "Marin City","City Name"] = "Marin"
df_2013.loc[df_2013["City Name"] == "Strawberry CDP (Marin County)","City Name"] = "North Marin"
df_2013.loc[df_2013["City Name"] == "Windsor Town","City Name"] = "Windsor"
df_2013.loc[df_2013["City Name"] == "Indian Wells","City Name"] = "Indian Wells Valley"
df_2013.loc[df_2013["City Name"] == "Rancho Calaveras","City Name"] = "Calaveras"

df_2013.loc[df_2013["City Name"] == "Bayview CDP (Humboldt County)","City Name"] = "Humboldt Bay"
df_2013.loc[df_2013["City Name"] == "East Los Angeles","City Name"] = "Los Angeles County Public Works Waterworks District 29"
df_2013.loc[df_2013["City Name"] == "Placerville","City Name"] = "Placer"
df_2013.loc[df_2013["City Name"] == "Nevada","City Name"] = "Nevada City"
df_2013.loc[df_2013["City Name"] == "Mission Hills","City Name"] = "Mission Springs"
df_2013.loc[df_2013["City Name"] == "Sunnyside-Tahoe City","City Name"] = "Tahoe City"
df_2013.loc[df_2013["City Name"] == "Amador City","City Name"] = "Amador Water Agency"
df_2013.loc[df_2013["City Name"] == "Truckee town","City Name"] = "Truckee-Donner"
df_2013.loc[df_2013["City Name"] == "Joshua Tree","City Name"] = "Joshua Basin"
df_2013.loc[df_2013["City Name"] == "Tuolumne City","City Name"] = "Tuolumne Utilities"
df_2013.loc[df_2013["City Name"] == "Amador City","City Name"] = "Amador Water Agency"
df_2013.loc[df_2013["City Name"] == "San Buenaventura (Ventura)","City Name"] = "San Buenaventura"

df_2013.loc[df_2013["City Name"] == "Castaic","City Name"] = "Castaic Lake Water Agency Santa Clarita Water Division"
df_2013.loc[df_2013["City Name"] == "Rio Vista","City Name"] = "Rio Vista, city of"
df_2013.loc[df_2013["City Name"] == "Soquel","City Name"] = "Soquel Creek"
df_2013.loc[df_2013["City Name"] == "Bear Creek","City Name"] = "Bear Gulch"
df_2013.loc[df_2013["City Name"] == "Hillsborough town","City Name"] = "Hillsborough"
df_2013.loc[df_2013["City Name"] == "North San Juan","City Name"] = "San Juan"
df_2013.loc[df_2013["City Name"] == "West Rancho Dominguez","City Name"] = "Dominguez"

df_2013.loc[df_2013["City Name"] == "Hermosa Beach","City Name"] = "Hermosa/Redondo"
df_2013.loc[df_2013["City Name"] == "West Rancho Dominguez","City Name"] = "Yucaipa Valley"
df_2013.loc[df_2013["City Name"] == "El Paso de Robles (Paso Robles)","City Name"] = "Paso Robles"
df_2013.loc[df_2013["City Name"] == "Palos Verdes Estates","City Name"] = "Palos Verdes"
df_2013.loc[df_2013["City Name"] == "Westlake Village","City Name"] = "Westlake"
df_2013.loc[df_2013["City Name"] == "Mammoth Lakes town","City Name"] = "Mammoth"

df_2013.loc[df_2013["City Name"] == "Marina","City Name"] = "Marina Coast"
df_2013.loc[df_2013["City Name"] == "Livermore","City Name"] = "Livermore "
df_2013.loc[df_2013["City Name"] == "La Crescenta-Montrose","City Name"] = "Crescenta Valley"
df_2013.loc[df_2013["City Name"] == "San Garbriel","City Name"] = "San Gabriel Valley"

In [680]:
#df_2010.to_csv("data_2010.csv")
#df_2011.to_csv("data_2011.csv")
#df_2012.to_csv("data_2012.csv")
df_2013.to_csv("data_2013.csv")

## Clean Diego Data

In [693]:
diego_list = []
with open("rationalberttwo (1).csv") as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        diego_list.append(row)        

In [694]:
diego_df = pd.DataFrame(diego_list[1:], columns = diego_list[0])
diego_df.drop("", 1, inplace= True)

suffix_to_exclude = ["  City of", ", City of", ", City of "," City of", " Municipal Water District",
                    " Public Utilities District", " Irrigation District", " Public Utility District"
                    "Water & Sanitary District", " Water & Sanitary District"" County Service Area 70",
                    " Services District", " Water District", " Municipal Utilities District",
                    " Community Service District", " Water Company",
                    " Department of Water and Power", " County", " Community", " Mutual",
                    " Water Department", " Public Utilities Commission", " Suburban", ", Town of",
                    " County Water Agency", " County Public Works Waterworks District 40",
                    " Public Utility District", " City of Public Works Department",
                    " Land and Water Association", " County Service Area 64",
                    " Water and Power Agency", "  Agency", " Water Service",
                    " Town of", "/Suburban", " Municipal Improvement District",
                    " Sanitation District / Oak Park Water Service",
                    " City of Division of Water Resources", " Land and",
                    " City of Public Works"," Fontana", ", Dept of Water & Power",
                    " District", " Water Authority", " County Waterworks District No. 8",
                    " County Waterworks District No 1", " Water Company Incorporated",
                    " Authority", "-Bell Gardens", " Water Service", " City of"]

prefix_to_exclude = ["City of ", "California Water Service Company ", "California Water Service Company ",
                     "Western Municipal Water District of ", "Golden State Water Company ",
                    "California-American Water Company ", "Suburban Water Systems ",
                    "Suburban Water Systems "]

In [695]:
cnt = 0
for name_index in range(len(diego_df)):
    for suffix in suffix_to_exclude:
        suffix_size = len(suffix)
        if diego_df.iloc[name_index,0][-suffix_size:] == suffix:
            diego_df.iloc[name_index,0] = diego_df.iloc[name_index,0][:-suffix_size]
            cnt += 1  
    for prefix in prefix_to_exclude:
        prefix_size = len(prefix)
        if diego_df.iloc[name_index,0][:prefix_size] == prefix:
            diego_df.iloc[name_index,0] = diego_df.iloc[name_index,0][prefix_size:]
            cnt += 1  
print "Number of treated elements: ",cnt

6521


In [696]:
diego_df.to_csv("diego_df.csv")

In [ ]:
'''
diego_df[diego_df["Supplier.Name"] == "Jurupa"] = "Jurupa Valley"
diego_df[diego_df["Supplier.Name"] == "Orange Vale"] = "Orangevale"
diego_df[diego_df["Supplier.Name"] == "Dublin San Ramon"] = "Dublin"
diego_df[diego_df["Supplier.Name"] == "El Dorado"] = "El Dorado Hills"
diego_df[diego_df["Supplier.Name"] == "San Lorenzo Valley"] = "San Lorenzo"
diego_df[diego_df["Supplier.Name"] == "South Tahoe"] = "South Lake Tahoe"
diego_df[diego_df["Supplier.Name"] == "Elsinore Valley"] = "Lake Elsinore"
diego_df[diego_df["Supplier.Name"] == "Coachella Valley"] = "Coachella"
diego_df[diego_df["Supplier.Name"] == "Contra Consta"] = "Contra Consta Center"
diego_df[diego_df["Supplier.Name"] == "Antelope Valley"] = "Antelope"
diego_df[diego_df["Supplier.Name"] == "Irvine Ranch"] = "Irvine"
diego_df[diego_df["Supplier.Name"] == "Rowland"] = "Rowland Heights"
diego_df[diego_df["Supplier.Name"] == "South Tahoe"] = "South Lake Tahoe"

diego_df[diego_df["Supplier.Name"] == "Hi-Desert"] = "Desert Center"
diego_df[diego_df["Supplier.Name"] == "Marin"] = "Marin City"
diego_df[diego_df["Supplier.Name"] == "North Marin "] = "Strawberry CDP (Marin County)"
diego_df[diego_df["Supplier.Name"] == "Windsor"] = "Windsor Town"
diego_df[diego_df["Supplier.Name"] == "Indian Wells Valley"] = "Indian Wells"
diego_df[diego_df["Supplier.Name"] == "Calaveras"] = "Rancho Calaveras"

diego_df[diego_df["Supplier.Name"] == "Humboldt Bay"] = "Bayview CDP (Humboldt County)"
diego_df[diego_df["Supplier.Name"] == "Los Angeles County Public Works Waterworks District 29"] = "East Los Angeles"
diego_df[diego_df["Supplier.Name"] == "Placer"] = "Placerville"
diego_df[diego_df["Supplier.Name"] == "Nevada City"] = "Nevada"
diego_df[diego_df["Supplier.Name"] == "Mission Springs"] = "Mission Hills"
diego_df[diego_df["Supplier.Name"] == "Tahoe City"] = "Sunnyside-Tahoe City"
diego_df[diego_df["Supplier.Name"] == "Amador Water Agency"] = "Amador City"
diego_df[diego_df["Supplier.Name"] == "Truckee-Donner"] = "Truckee town"
diego_df[diego_df["Supplier.Name"] == "Joshua Basin"] = "Joshua Tree"
diego_df[diego_df["Supplier.Name"] == "Tuolumne Utilities"] = "Tuolumne City"
diego_df[diego_df["Supplier.Name"] == "San Buenaventura"] = "San Buenaventura (Ventura)"

diego_df[diego_df["Supplier.Name"] == "Castaic Lake Water Agency Santa Clarita Water Division"] = "Castaic"
diego_df[diego_df["Supplier.Name"] == "Rio Vista, city of"] = "Rio Vista"
diego_df[diego_df["Supplier.Name"] == "Soquel Creek"] = "Soquel"
diego_df[diego_df["Supplier.Name"] == "Bear Gulch"] = "Bear Creek"
diego_df[diego_df["Supplier.Name"] == "Hillsborough"] = "Hillsborough town"
diego_df[diego_df["Supplier.Name"] == "San Juan"] = "North San Juan"
diego_df[diego_df["Supplier.Name"] == "Dominguez"] = "West Rancho Dominguez"

diego_df[diego_df["Supplier.Name"] == "Marina Coast"] = "Marina"
diego_df[diego_df["Supplier.Name"] == "Livermore "] = "Livermore"
diego_df[diego_df["Supplier.Name"] == "Crescenta Valley"] = "La Crescenta-Montrose"
diego_df[diego_df["Supplier.Name"] == "San Gabriel Valley"] = "San Garbriel"

diego_df[diego_df["Supplier.Name"] == "Hermosa/Redondo"] = "Hermosa Beacg"
diego_df[diego_df["Supplier.Name"] == "Yucaipa Valley"] = "Yucaipa"
diego_df[diego_df["Supplier.Name"] == "Paso Robles"] = "El Paso de Robles (Paso Robles)"
diego_df[diego_df["Supplier.Name"] == "Palos Verdes"] = "Palos Verdes Estates"
diego_df[diego_df["Supplier.Name"] == "Westlake"] = "Westlake Village"
diego_df[diego_df["Supplier.Name"] == "Mammoth"] = "Mammoth Lakes town"
'''